In [ ]:
# Data 205 - Capstone in Data Science
# Jennifer Paraboschi     Fall 2020
# Inputs: data sets from dataMontgomery
# including Alcohol Beverage Licensing Violations

In [ ]:
# Import packages. 
# Pull in API data set as json.
import json
import numpy as np
import pandas as pd
import re
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import plotly    # I had to pip install plotly on the cmd line to get plotly to work
import plotly.express as px

In [ ]:
violations_data = pd.DataFrame(requests.get("https://data.montgomerycountymd.gov/resource/4tja-rkhg.json").json())
violations_data.head()
#pd.read_json("https://....")    #alternative way to read the data

In [ ]:
print(violations_data)

In [ ]:
violations_data.info()

In [ ]:
# need to change the dates so they are in a useable format.
violations_data.violationdate = pd.to_datetime(violations_data.violationdate)
violations_data.dispositiondate = pd.to_datetime(violations_data.dispositiondate)

In [ ]:
violations_data.info()

In [ ]:
violations_data.describe(include="all")

In [ ]:
# There are a few missing values for disposition and disposition date (counts are fewer than total count in some columns).
# The most frequent violation is for sale to minor.

In [ ]:
violations_data.isnull().sum()
# This gives the number of null values for each var.

In [ ]:
# Other EDA from https://www.youtube.com/watch?v=-o3AxdVcUtQ
violations_data.nunique()
# This gives the number of unique values for each variable

In [ ]:
# Provide a list of the violations.
violations_data.violation.unique()

In [ ]:
# From this video https://www.youtube.com/watch?v=5NcbVYhQJvw
# count by category crosstab
violations_dist = violations_data.groupby("violation").size()

In [ ]:
# Get counts of the number of violations at each violation type.
violations_data["violation"].value_counts().sort_values(ascending=False)

In [ ]:
violations_dist.plot(title="Distribution of Violations")

In [ ]:
# This is not helpful unless I can clean it up. 
# Put the violations into groups/categories. 

In [ ]:
# Get counts of the number of violations at each facility name.
pd.set_option("display.max_rows", None)
violations_data["facilityname"].value_counts().sort_values(ascending=False)

In [ ]:
# Gaithersburg Supermarket has had the greatest number of violations. 
violations_data[violations_data.facilityname == "GAITHERSBURG SUPERMARKET"]

In [ ]:
"""
Explore the penalties for the violations
"""

In [ ]:
# The disposition var inconsistently contains a $ amount of the penalty with inconsistent formatting. 
# Use a regex to extract the penalty values.
violations_data["penalty"]=violations_data["disposition"].str.extract(r"((?<=\$)\d+)")
# This regex looks for the dollar sign (\$), then pulls all the digits (\d+) unlimited times (to the end).

In [ ]:
violations_data["penalty"] # checking that the penalty amounts pulled in correctly.

In [ ]:
violations_data["disposition"] # checking that the disposition column remained unchanged. 

In [ ]:
violations_data.isnull().sum() # provides a count of nulls for each var.

In [ ]:
# There are 51 records with no penalty value (nulls). 
# Some records did not result in a penalty being assessed.
# Need to provide a value (0) for the missing penalty amounts. 

In [ ]:
# Replace missing penalty values with 0.
violations_data["penalty"]=violations_data["penalty"].fillna(0)
# change the penalty type to an integer.
violations_data["penalty"]=violations_data["penalty"].astype(int)

In [ ]:
violations_data

In [ ]:
violations_data.info()

In [ ]:
# Get counts of the number of violations at each penalty amount.
violations_data["penalty"].value_counts().sort_values(ascending=False)

In [ ]:
# Most of the violations are for $1,000, $100, or $500. 
# Look at the max value or the facility that received the highest penalty amount.
violations_data[violations_data.penalty == violations_data.penalty.max()]

In [ ]:
# Silver Spring Hilton Hotel had the violation with the highest penalty. 
violations_data[violations_data.facilityname == "SILVER SPRING HILTON HOTEL"]

In [ ]:
# Explore distribution of the penalty amounts.
sns.distplot(violations_data["penalty"])  # or can add , bins = 20)]

In [ ]:
sns.catplot(x="penalty", kind="box", data=violations_data)

In [ ]:
sns.set()
_ = plt.hist(violations_data['penalty'])
_ = plt.xlabel('Amount')
_ = plt.ylabel('Count of Penalties at that Amount')
plt.show()

In [ ]:
violations_data.describe(include="all")

In [ ]:
# From above - counts of violations. These are the top 5 most frequent violations.
"""
SALE TO MINOR                                                                                      381
ALCOHOL AWARENESS CERTIFIED PERSON NOT ON PREMISES                                                 173
6.1 SALES OR SERVICE TO MINORS/CONSUMPTION OR POSSESSION OF MINORS                                  66
EMPLOYEE RECORDS NOT AVAILABLE WHEN REQUESTED                                                       64
LICENSE NOT PROPERLY DISPLAYED                                                                      57
"""

# Pull top 5 most frequent violations into a subset
top_5_violations = violations_data[violations_data["violation"].isin(["SALE TO MINOR","ALCOHOL AWARENESS CERTIFIED PERSON NOT ON PREMISES","6.1 SALES OR SERVICE TO MINORS/CONSUMPTION OR POSSESSION OF MINORS","EMPLOYEE RECORDS NOT AVAILABLE WHEN REQUESTED","LICENSE NOT PROPERLY DISPLAYED"])]


In [ ]:
# count by category crosstab
violations_dist_top5 = top_5_violations.groupby("violation").size()

In [ ]:
violations_dist_top5.plot(title="Distribution of Top Violations")


In [ ]:

# Perhaps look at average amount of penalty for each kind of violation?
# Maybe bar charts of violation amounts by type? 

In [ ]:
# box plot categorical vars
box_violations=sns.boxplot(x="violation", y="penalty", data=violations_data)

In [ ]:
"""
Explore violations by zip code
"""

In [ ]:
# Copy the license violations zip codes into a separate var.
# Use a regular expression to pull out the zip codes.
violations_data["zip"]=violations_data["address"].str.extract(r"((?<=.)\d{5})")
# positive lookbehind, matches to digits exactly 5 long

violations_data["zip"]=violations_data["zip"].astype(int)
violations_data

In [ ]:
pd.set_option("display.max_colwidth", None)
addr=violations_data["address"].astype(str)
mask=addr.str.slice(-5,-4,1) == "-"

In [ ]:
# QA the slicing for zip.
violations_data.loc[mask, ["address","zip"]]

In [ ]:
violations_data["address"]

In [ ]:
violations_data

In [ ]:
# Now I have the zip codes of the Alcohol Beverage License Violations in the var "zip".
# Do frequency of violations by zip code.

In [ ]:
# Get counts of the number of violations by zip.
violations_data["zip"].value_counts().sort_values(ascending=False)

In [ ]:
# Now the zips are all 5-digits
# Bethesda (20814) and Rockville (20852) have the 2 highest frequencies. 

In [ ]:
violations_data.sort_values(by=["zip", "facilityname"])

sns.set()
_ = plt.hist(violations_data["zip"])
_ = plt.xlabel('Zip')
_ = plt.ylabel('Count of Violations')
plt.show()

In [ ]:
# This plots the top 10 most frequent zip codes. 
violations_data["zip"].value_counts()[:10].plot(kind="barh")


In [ ]:
# This has the potential to be interesting as I recognize some of the zip codes/areas. 
# I'll come back to this.

In [ ]:
### This works but there are too many zip codes to the display is too small to read.

# facet grid of violation amounts by zip code
#grid_viol_zip = sns.FacetGrid(violations_data, col="zip")
#grid_viol_zip.map(plt.hist, "penalty")

In [ ]:
"""
Explore the violations involving minors.
"""

In [ ]:
# Investigate frequencies by types of violations. 
# Group the sale to minor and 6.1 sales or service to minors. 

In [ ]:
# Groupby() function to pull the 2 violations together.
violations_minors = violations_data.groupby("violation").get_group("SALE TO MINOR" or "6.1 SALES OR SERVICE TO MINORS/CONSUMPTION OR POSSESSION OF MINORS")

In [ ]:
violations_minors

In [ ]:
type(violations_minors)

In [ ]:
violations_minors.describe(include="all")

In [ ]:
# This plots top 20 zip codes for the violations involving minors.
violations_minors["zip"].value_counts()[:20].plot(kind="barh")

In [ ]:
violations_minors["zip"].value_counts()[:10].plot(kind="barh")

In [ ]:
# Since violations involving minors are the most frequent violation overall...
# the plots of the full data set and the violations involving minors data set are very similar.

# Use a stacked bar chart. 
"""
import plotly.express as px

df = px.data.iris()

fig = px.bar(df, x="sepal_width", y="sepal_length", color="species",
			hover_data=['petal_width'], barmode = 'stack')

fig.show()


"""

In [ ]:
"""
Explore the High Schools
"""

In [ ]:
# I may try to overlay the high school locations with areas of highest crime and/or alcohol violations. 

In [ ]:
# Import the public high schools data set.
schools_data = pd.DataFrame(requests.get("https://data.montgomerycountymd.gov/resource/772q-4wm8.json").json())
schools_data.head()

In [ ]:
# Drop the unnecessary columns (i.e., category, elementary/middle schools, phone and url).

In [ ]:
high_schools=schools_data[schools_data["category"] == "HIGH SCHOOLS"]
cols_drop=["category","phone","url"]
high_schools.drop(cols_drop, inplace=True, axis=1)
print(high_schools)

In [ ]:
# Map the high school locations.

In [ ]:
# Import the plotly express package

import plotly.express as px

fig_schools = px.scatter_geo(high_schools, 
                     lon="longitude", 
                     lat="latitude",
                     # choose the map chart's projection
                     projection="albers usa",
                     center=dict(lon=-77.14, lat=39.098),
                     # columns which is in bold in the pop up
                     hover_name = "school_name",
                     # format of the popup not to display these columns' data
                     hover_data = {"longitude": False, "latitude": False})
fig_schools.show()



In [ ]:
# I'm having trouble zooming in on this map. I decided to use a different map (below) that includes streets.

In [ ]:
# Follow instructions from here: https://plotly.com/python/mapbox-layers/#openstreetmap-tiles-no-token-needed
high_schools['latitude']=high_schools['latitude'].astype(float)
high_schools['longitude']=high_schools['longitude'].astype(float)

fig_schools_map = px.scatter_mapbox(high_schools, lat="latitude", lon="longitude", hover_name="school_name", zoom=9, 
                         hover_data={"latitude":False, "longitude":False})
fig_schools_map.update_layout(mapbox_style="open-street-map")
fig_schools_map.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_schools_map.show()


In [ ]:
# Note To Self: I don't know what to do about the warnings (above)
    # A value is trying to be set on a copy of a slice from a DataFrame.
    # Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
# Note To Self: While this map is OK, I can't decide what would make it better. 
# I was not able to make the points/circles bigger.
# I tried zooming in closer but then not all of the high schools show up.
# I tried to get the city to display as well as the HS name but was not able to get this to work.

# I'll try to map the violations and schools using Tableau.

In [ ]:
"""
Explore mapping the violations.
"""

In [ ]:
# Note To Self: I was able to clean the violation addresses in excel, upload to geocodio, then use that csv to map. 
# However, I want to be able to do this directly from the dataMontgomery API. 

# I tried again for the geolocation using geopy (below) following these instructions from towardsdatascience.com.
#  https://towardsdatascience.com/pythons-geocoding-convert-a-list-of-addresses-into-a-map-f522ef513fd6
# I think there is a limit to how many times I can use the geolocater/site though. Not sure about that. 

In [ ]:
# Import the violations data set (I don't need to do this repeatedly but sometimes I am picking up here when I start again)
violations_data = pd.DataFrame(requests.get("https://data.montgomerycountymd.gov/resource/4tja-rkhg.json").json())
violations_data.head()

In [ ]:
# I had to pip install geopy on the cmd line to get this to work.
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="sample app")
# Apply geolocator.geocode to the address column
violations_data["location"]=violations_data["address"].apply(geolocator.geocode)

In [ ]:
violations_data["point"]=violations_data["location"].apply(lambda loc: tuple(loc.point) if loc else None)

In [ ]:
# Then pull the values into 3 diff vars (the 3rd one is altitude)

In [ ]:
violations_data[["latitude", "longitude", "altitude"]] = pd.DataFrame(violations_data["point"].to_list(), index=violations_data.index)

In [ ]:
# Map the locations of the violations
fig_violations_map = px.scatter_mapbox(violations_data, lat="latitude", lon="longitude", hover_name="facilityname", zoom=9, 
                         hover_data={"latitude":False, "longitude":False})
fig_violations_map.update_layout(mapbox_style="open-street-map")
fig_violations_map.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_violations_map.show()


In [ ]:
"""
Explore CRASH data
"""

In [ ]:
# Pull in the Crash Incidents data 
# Trying to narrow down by alcohol activity because the dataset is so big.
selections = {"Driver Substance Abuse": "ALCOHOL PRESENT"}

crash_incidents_data = pd.DataFrame(requests.get("https://data.montgomerycountymd.gov/resource/bhju-22kf.json",params=selections).json())
crash_incidents_data.head()

In [ ]:
crash_incidents_data.shape

In [ ]:
crash_incidents_data.describe(include="all")

In [ ]:
"""
Explore CRIME data
"""

In [ ]:
# Pull in the Crimes data 
# Trying to narrow by crimes against society and then crime2 for alcohol violations. 
# looking at the crimes data, there are only 8 alcohol-related crimes in the crimes data set.
# under Crime Name 2  Drunkenness,  Driving Under the Influence,  Liquor Law Violations
#select_crime = {"crimename1": "Crime Against Society"}

crime_data = pd.read_csv("crime.csv")
crime_data.head()

In [ ]:
crime_data.info()

In [ ]:
# Drop unnecessary columns
cols_to_drop=["Offence Code","CR Number","Dispatch Date / Time","Victims","Agency","Place","Sector","Beat","PRA","Address Number","Street Prefix","Street Name","Street Suffix","Street Type","End_Date_Time"]
crime_data.drop(cols_to_drop, inplace=True, axis=1)
crime_data.info()

In [ ]:
crime_data.describe(include="all")

In [ ]:
# There are 3,187 records with blank zip codes.
# There are another 200+ with incorrect zips (wrong number of digits and/or not in Montgomery County).
# Replace missing zip codes values with 0.
crime_data["Zip Code"]=crime_data["Zip Code"].fillna(0)
# change the zip code type to an integer.
crime_data["Zip Code"]=crime_data["Zip Code"].astype(int)

# This plots top 20 zip codes for the crimes.
crime_data["Zip Code"].value_counts()[:20].plot(kind="barh")

In [ ]:
"""
Explore Population data by zip code
"""

In [ ]:
# Pull population data from this site https://worldpopulationreview.com/zips/maryland
# This is the csv link:  blob:https://worldpopulationreview.com/00124d35-9d90-48ad-973f-a3eaddcbe13e 
# this is the json link:  blob:https://worldpopulationreview.com/eaa13b61-5379-49d2-a077-3d611a223c7b
# I downloaded MD counties with populations by zip code so I can select Montgomery County. 
# The site says it reflects 2020 population data.
# blob:https://worldpopulationreview.com/087018d8-c25a-44cf-a6d9-cb479a108878
  #      blob:https://worldpopulationreview.com/507fea65-9f1a-483d-89a4-b640e1b3e9bb

In [ ]:
pop_zip_codes = pd.read_csv('PopulationZip.csv')
pop_zip_codes.head()

In [ ]:
pop_zip_codes.info()

In [ ]:
# groupby() to pull only Montgomery County zips.
pop_zip_mont = pop_zip_codes.groupby("county").get_group("Montgomery")
pop_zip_mont

In [ ]:
pop_zip_mont.info()

In [ ]:
# Get frequency of all crimes by zip code. (crimes data set has zip code column) 
# Need to calculate violations per population by zip code and crimes per population by zip code. 
# Compare alcohol violations by zip code with crimes by zip code. 

In [ ]:
# zip codes with high crime rates and zip codes with high ABS licenses and/or violations = correlation? 
# not just look at major crimes but also petty crimes. 

# Do this by taking ABS licenses (or violations) by population by zip code? 
# Then crimes by population by zip code? 